# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'local', 'travel_db', 'uk_food']


In [5]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [6]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [9]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'Reach Court Farm',
 'AddressLine2': 'Reach Road',
 'AddressLine3': 'St Margarets At Cliffe',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Barn',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.77622109052,
 'FHRSID': 429982,
 'LocalAuthorityBusinessID': 'PI/000043978',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6AQ',
 'RatingDate': '2017-09-21T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6553cfbca00fff7b50f12f70'),
 'geocode': {'latitude': '51.147379', 'longitude': '1.371115'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/429982',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate'

In [10]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [12]:
# Create a dictionary for the new restaurant data
new_halal_rest = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [13]:
# Insert the new restaurant into the collection
establishments.insert_one(new_halal_rest)

In [14]:
# Check that the new restaurant was inserted
query = {'BusinessName': 'Penang Flavours', 'LocalAuthorityCode':'511'}
new_rest = establishments.find(query)
for restaurant in new_rest:
    pprint(restaurant)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6553d0e8a2bc6ee09eaba9ef'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [26]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType':"Restaurant/Cafe/Canteen"}
fields = {'BusinessTypeID':1, 'BusinessType':1}

# Capture the results with a variable
results = establishments.find(query, fields)

pprint(results[0])

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('6553cfbca00fff7b50f12f70')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [28]:
# Update the new restaurant with the correct BusinessTypeID
query = {'BusinessTypeID': 1} 
results = establishments.update_one({'BusinessName': 'Penang Flavours'},{'$set':{"BusinessTypeID":1}})
results

In [29]:
# Confirm that the new restaurant was updated
query = {'BusinessName': 'Penang Flavours'}
fields = {'BusinessName':1, 'BusinessTypeID':1}

restaurant_check = establishments.find_one(query, fields)
pprint(restaurant_check)

{'BusinessName': 'Penang Flavours',
 'BusinessTypeID': 1,
 '_id': ObjectId('6553d0e8a2bc6ee09eaba9ef')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [30]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName': 'Dover'}

# Print the number of results
print("Number of establishments located in Dover:", establishments.count_documents(query))

Number of establishments located in Dover: 994


In [31]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({'LocalAuthorityName': 'Dover'})

In [32]:
# Check if any remaining documents include Dover
query = {'LocalAuthorityName': 'Dover'}

# Print the number of results
print("Number of documents that include Dover:", establishments.count_documents(query))

Number of documents that include Dover: 0


In [35]:
# Check that other documents remain with 'find_one'
query = {}
fields = {'BusinessName'}

establishments.find_one(query, fields)

{'_id': ObjectId('6553cfbca00fff7b50f13250'),
 'BusinessName': 'Wear Bay Bowls Club'}

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [36]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [ {'$set':{ "geocode.latitude" : {'$toDouble': "$geocode.latitude"}}} ])
establishments.update_many({}, [ {'$set':{ "geocode.longitude" : {'$toDouble': "$geocode.longitude"}}} ])

Use `update_many` to convert `RatingValue` to integer numbers.

In [37]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [38]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [ {'$set':{ "RatingValue" : {'$toInt': "$RatingValue"}}} ])

In [39]:
# Check that the coordinates and rating value are now numbers
query = {}
fields = {'geocode':1, 'RatingValue':1}

establishments.find_one(query, fields)

{'_id': ObjectId('6553cfbca00fff7b50f13250'),
 'RatingValue': 4,
 'geocode': {'longitude': 1.196408, 'latitude': 51.086058}}